# Information Retrieval in High Dimensional Data
## Lab 4

|     |     |
| --- | --- |
| **Name:** | Uzair Akbar |
| **Matriculation Number:** | 03697290 |
| **E-mail:** | [uzair.akbar@tum.de](mailto:uzair.akbar@tum.de) |

## Logistic Regression
### Task 1
Recall the data processing routines from the last lab course. The following excercises build on top of the extracted feature representations, but instead of the prebuilt classifier, we want to implement logistic regression by hand, i.e. by minimizing $F(\mathbf{w})$ from Chapter 3 in the lecture notes. To this end, make sure, that the variables `train`, `test`, `train_data_features` and `test_data_features` are loaded to your IPython shell.

In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

#%% download nltk stopwords
# import nltk
# ntlk.download('stopwords')

# load stopwords
stops = set(stopwords.words('english'))

# function for preprocessing the data
def review_prepro(data, remove_stopwords=False):
    # remove HTML tags
    review_text = BeautifulSoup(data, 'lxml').get_text()
    # remove non-letters and numbers
    letters_only = re.sub( '[^a-zA-Z]',
                          ' ',
                          review_text )
    # make all characters lower case and split the documents into single words
    words = letters_only.lower().split()
    
    if remove_stopwords:
        # remove stop words
        meaningful_words = [ w for w in words if not w in stops ]
        # return concatenated single string
        return ' '.join(meaningful_words)
    else:
        # or don't and concatenate to single string
        return ' '.join(words)

# load data as pandas dataframe
train = pd.read_csv('labeledTrainData.tsv', 
                    header=0,
                    delimiter="\t", 
                    quoting=3 )

test = pd.read_csv('labeledTestData.tsv', 
                   header=0,
                   delimiter="\t",
                   quoting=3 )


# preprocess train and test data
num_reviews = train['review'].size

clean_train_reviews = []
for i in range(num_reviews):
  #  if (i+1)%1000 == 0:
    #    print('Review {} of {}\n'.format(i+1, num_reviews))
    clean_train_reviews.append( review_prepro(train['review'][i], remove_stopwords=True) )
    
num_test_reviews = test['review'].size

clean_test_reviews = []
for i in range(num_test_reviews):
    #if (i+1)%1000 == 0:
     #   print('Review {} of {}\n'.format(i+1, num_test_reviews))
    clean_test_reviews.append( review_prepro(test['review'][i], remove_stopwords=True) )
    

#%% create BoW
# Documentation:
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
vectorizer = CountVectorizer(analyzer = 'word',
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = stops,
                             max_features = 5000)

# fit the vectorizer and return transformed reviews
vectorizer.fit(clean_train_reviews)
train_data_features = vectorizer.transform(clean_train_reviews)
clean_train_reviews=None 
# convert to numpy array
train_data_features = train_data_features.toarray()

# create BoW representation of test data
test_data_features = vectorizer.transform(clean_test_reviews)
clean_test_reviews=None
test_data_features = test_data_features.toarray()

**a)** Write a PYTHON function `logistic_gradient` that expects a training set matrix `X_train`, a ground truth label vector `y_train` and a current weight vector `w` as its input and returns the gradient `g` of the negative log-likelihood function of the logistic regression. Refer to the lecture notes for the mathematical definition.

In [6]:
import numpy as np

def sigmoid(x):
    # https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    z = np.exp(-np.abs(x))
    return np.where(x>=0.0,1.0/(1.0+z),z/(1.0+z))

def logistic_gradient(X_train,y_train,w,reg=0.0):
    # Rows are variables, Columns are samples
    g=np.dot(X_train*np.expand_dims(y_train,0),-sigmoid(-np.dot(X_train.T, w)*y_train))/X_train.shape[1]+2*reg*w
    return g

**b)** Write a PYTHON function `find_w` that expects a training set matrix `X_train`, a ground truth label vector `y_train`, a step size `alpha` and a maximum iteration number `max_it` that determines the optimal logistic regression weight vector `w_star` by performing gradient descent, i.e. calling `logistic_gradient` in each iteration. Make sure to include the affine offset $w_0=b$ into your model.

In [10]:
def find_w(X_train, y_train, alpha, n_epochs, n_minibatch, reg=0.0,w_init=None):
    X_offs=np.ones((X_train.shape[0]+1,X_train.shape[1]))
    X_offs[1:,:]=X_train
    w_init=np.ones((X_offs.shape[0]))
    if w_init is not None:
        w=w_init
    loss=np.inf
    for epoch in range(n_epochs):
        zzz=-np.log(sigmoid(np.expand_dims(y_train,0)*np.dot(w.T,X_offs)))
        loss=np.sum(zzz)/X_offs.shape[1]+reg*np.sum(w**2)
        print('Epoch:', epoch+1, ' Current loss:', loss)
        rp=np.random.permutation(X_offs.shape[1])
        for it in range(X_offs.shape[1]//n_minibatch):
            delta_w=logistic_gradient(X_offs[:,rp[it*n_minibatch:(it+1)*n_minibatch]],
                                      y_train[rp[it*n_minibatch:(it+1)*n_minibatch]],w,reg)*alpha
            w-=np.array(delta_w)
        w_star=w
    return w_star

**c)** The dataset at hand is quite large. Applying standard gradient descent could likely lead to Python throwing a `MemoryError` exception. To avoid this, we will employ a variation of *stochastic gradient descent* that has been proven successful in training deep neural networks. In *minibatch learning*, each iteration of the algorithm is replaced by a so-called *epoch*. In each epoch the training set is divided randomly into subsets of equal size, the minibatches. For each minibatch, the gradient is computed and applied only with respect to the samples in the minibatch. An epoch is finished when the gradient step was performed for each minibatch. Modify `find_w` such that it is capable of mini-batch learning. You will need to replace `max_it` by `n_epochs` and and add the parameter `n_minibatch` to your function definition. Note: take care of normalization of the gradient and the loss function.

**d)** Write a function `classify_log` that expects a weight vector `w` and a test set matrix `X_test` and classifies the samples in `X_test` via logistic regression, returning a label vector `y_test`. Test your implementation of `find_w` and`classify_log` on `train_data_features` and `test_data_features` with 10 epochs, minibatches of size 100 and step size `alpha=1`.

In [13]:
from sklearn.metrics import roc_auc_score as AUC

def classify_log(w, X_test):
    w0=w[0]
    y_test=sigmoid(w0+np.dot(X_test.T,w[1:]))
    return y_test

#Testing implementation
y_train=train['sentiment'].values
y_test=test['sentiment'].values
w=find_w(train_data_features.T,(y_train-0.5)*2,1,10,100)
y_pred=classify_log(w,test_data_features.T)
auc = AUC( y_test, y_pred )
print('AUC score after 10 epochs:',auc)


Epoch: 1  Current loss: 48.15060140614601
Epoch: 2  Current loss: 1.6211494936806483
Epoch: 3  Current loss: 0.9489240091513209
Epoch: 4  Current loss: 0.6715706172277032
Epoch: 5  Current loss: 0.5424227522524626
Epoch: 6  Current loss: 0.43959397977810716
Epoch: 7  Current loss: 0.3736328258829605
Epoch: 8  Current loss: 0.32666368726381906
Epoch: 9  Current loss: 0.2948693046337706
Epoch: 10  Current loss: 0.27121470376342915
AUC score after 10 epochs: 0.9141645142775147


**e)** Logistic regression is prone to *overfitting*. To prevent this, *regularizers* can be used. Adjust your implementation in such a way that instead of minimizing $F(\mathbf{w})$, it minimizes the term
    	\begin{equation}
    	F(\mathbf{w})+\lambda\|\mathbf{w}\|^2,
    	\end{equation}
    	where $\lambda$ is a non-negative regularization parameter. Test your implementation with $\lambda=10^{-3}$.  Did the results improve? Why/why not?

In [14]:
w=find_w(train_data_features.T,(y_train-0.5)*2,1,10,100,1e-3)
y_pred=classify_log(w,test_data_features.T)
auc = AUC( y_test, y_pred )
print('AUC score after 10 epochs:',auc)

Epoch: 1  Current loss: 53.15160140614601
Epoch: 2  Current loss: 2.3726516823913357
Epoch: 3  Current loss: 0.9779694421855039
Epoch: 4  Current loss: 0.5246741941074616
Epoch: 5  Current loss: 0.523786207255704
Epoch: 6  Current loss: 0.45656475070242475
Epoch: 7  Current loss: 0.47913889545015886
Epoch: 8  Current loss: 0.3297247518528958
Epoch: 9  Current loss: 0.3373310028559831
Epoch: 10  Current loss: 0.33449671506682555
AUC score after 10 epochs: 0.9387975191152655
